In [1]:
import numpy as np
import random
from helpers import *
from costs import *
from gradient_descent import *
from stochastic_gradient_descent import *

random.seed(10)

In [19]:
y, x, ids = load_csv_data(data_path="datas/train.csv", sub_sample=False)


In [20]:
# Set to 0 all column containing -999
def remove_NaN(x):
    columns_with_NaN = set("")
    for row in x:
        for i,feature in enumerate(row):
            if feature == -999:
                columns_with_NaN.add(i)

    for col in columns_with_NaN:
        x[:, col] = 0
        
    return x
        
x = remove_NaN(x)


In [21]:
def normalize(x):
    return (x - x.mean(axis=0)) / (x.std(axis=0) + 0.0000000001)
x = normalize(x)
x

array([[ 0.        ,  0.06833197,  0.40768027, ...,  0.        ,
         0.        ,  0.4125105 ],
       [ 0.        ,  0.55250482,  0.54013641, ...,  0.        ,
         0.        , -0.27381996],
       [ 0.        ,  3.19515553,  1.09655998, ...,  0.        ,
         0.        , -0.29396985],
       ..., 
       [ 0.        ,  0.31931645, -0.13086367, ...,  0.        ,
         0.        , -0.31701723],
       [ 0.        , -0.84532397, -0.30297338, ...,  0.        ,
         0.        , -0.74543941],
       [ 0.        ,  0.66533608, -0.25352276, ...,  0.        ,
         0.        , -0.74543941]])

In [9]:
w_init = np.random.rand(x.shape[1])
w, loss = least_squares_GD(y, x, w_init, max_iters=1000, gamma=0.02)

Gradient Descent(0/999): loss=7.803076073096768		0.549992
Gradient Descent(1/999): loss=6.690601548708		0.545052
Gradient Descent(2/999): loss=5.788049060693789		0.540264
Gradient Descent(3/999): loss=5.05234115373521		0.535732
Gradient Descent(4/999): loss=4.449503404070839		0.531456
Gradient Descent(5/999): loss=3.952720587870469		0.527588
Gradient Descent(6/999): loss=3.540809526715271		0.523948
Gradient Descent(7/999): loss=3.1970191619039596		0.520784
Gradient Descent(8/999): loss=2.9080876220051133		0.517764
Gradient Descent(9/999): loss=2.6635011331479013		0.514996
Gradient Descent(10/999): loss=2.454911465838178		0.512612
Gradient Descent(11/999): loss=2.275677911902412		0.510888
Gradient Descent(12/999): loss=2.120507087186857		0.509836
Gradient Descent(13/999): loss=1.9851695890993455		0.508716
Gradient Descent(14/999): loss=1.8662770399773092		0.50792
Gradient Descent(15/999): loss=1.761106582191091		0.507508
Gradient Descent(16/999): loss=1.667462666589086		0.507144
Gradien

Gradient Descent(138/999): loss=0.43281107306019145		0.68302
Gradient Descent(139/999): loss=0.43241691800320947		0.683432
Gradient Descent(140/999): loss=0.43203224247113187		0.683888
Gradient Descent(141/999): loss=0.43165674685381655		0.684224
Gradient Descent(142/999): loss=0.4312901425814184		0.684604
Gradient Descent(143/999): loss=0.4309321516654896		0.684952
Gradient Descent(144/999): loss=0.43058250626131805		0.685364
Gradient Descent(145/999): loss=0.43024094825041287		0.68586
Gradient Descent(146/999): loss=0.42990722884211696		0.6862
Gradient Descent(147/999): loss=0.4295811081933713		0.686496
Gradient Descent(148/999): loss=0.4292623550457229		0.686884
Gradient Descent(149/999): loss=0.4289507463787094		0.687328
Gradient Descent(150/999): loss=0.4286460670788036		0.687604
Gradient Descent(151/999): loss=0.4283481096231492		0.687808
Gradient Descent(152/999): loss=0.42805667377735585		0.688144
Gradient Descent(153/999): loss=0.4277715663066649		0.688548
Gradient Descent(154

Gradient Descent(273/999): loss=0.413275893431983		0.7029
Gradient Descent(274/999): loss=0.41322253213207305		0.702936
Gradient Descent(275/999): loss=0.41316960882890513		0.702984
Gradient Descent(276/999): loss=0.4131171169368069		0.703048
Gradient Descent(277/999): loss=0.41306505000733096		0.703108
Gradient Descent(278/999): loss=0.41301340172587814		0.703128
Gradient Descent(279/999): loss=0.4129621659084115		0.703116
Gradient Descent(280/999): loss=0.4129113364982647		0.70314
Gradient Descent(281/999): loss=0.41286090756303695		0.703196
Gradient Descent(282/999): loss=0.4128108732915746		0.703256
Gradient Descent(283/999): loss=0.41276122799103626		0.70324
Gradient Descent(284/999): loss=0.4127119660840367		0.70326
Gradient Descent(285/999): loss=0.412663082105872		0.703296
Gradient Descent(286/999): loss=0.4126145707018169		0.703344
Gradient Descent(287/999): loss=0.4125664266244984		0.703396
Gradient Descent(288/999): loss=0.4125186447313395		0.703496
Gradient Descent(289/999)

Gradient Descent(409/999): loss=0.408460615030726		0.7062
Gradient Descent(410/999): loss=0.40843654101182514		0.706224
Gradient Descent(411/999): loss=0.40841257423227656		0.706256
Gradient Descent(412/999): loss=0.40838871396919185		0.706236
Gradient Descent(413/999): loss=0.4083649595081766		0.706252
Gradient Descent(414/999): loss=0.408341310143189		0.706236
Gradient Descent(415/999): loss=0.4083177651764011		0.706252
Gradient Descent(416/999): loss=0.4082943239180618		0.706272
Gradient Descent(417/999): loss=0.4082709856863642		0.706252
Gradient Descent(418/999): loss=0.4082477498073145		0.706244
Gradient Descent(419/999): loss=0.40822461561460255		0.706224
Gradient Descent(420/999): loss=0.408201582449477		0.706268
Gradient Descent(421/999): loss=0.408178649660621		0.706248
Gradient Descent(422/999): loss=0.4081558166040311		0.706236
Gradient Descent(423/999): loss=0.40813308264289894		0.706256
Gradient Descent(424/999): loss=0.4081104471474935		0.706276
Gradient Descent(425/999)

Gradient Descent(543/999): loss=0.40598279002749454		0.70728
Gradient Descent(544/999): loss=0.4059687804145382		0.707284
Gradient Descent(545/999): loss=0.40595482371375946		0.707288
Gradient Descent(546/999): loss=0.4059409196952569		0.707308
Gradient Descent(547/999): loss=0.4059270681305153		0.707304
Gradient Descent(548/999): loss=0.4059132687923931		0.707284
Gradient Descent(549/999): loss=0.40589952145510716		0.707284
Gradient Descent(550/999): loss=0.40588582589422		0.707292
Gradient Descent(551/999): loss=0.40587218188662716		0.707296
Gradient Descent(552/999): loss=0.4058585892105425		0.70734
Gradient Descent(553/999): loss=0.4058450476454862		0.707328
Gradient Descent(554/999): loss=0.40583155697227224		0.707324
Gradient Descent(555/999): loss=0.40581811697299536		0.707336
Gradient Descent(556/999): loss=0.40580472743101886		0.707344
Gradient Descent(557/999): loss=0.40579138813096255		0.707328
Gradient Descent(558/999): loss=0.40577809885869026		0.707312
Gradient Descent(55

Gradient Descent(677/999): loss=0.40450113510198077		0.70784
Gradient Descent(678/999): loss=0.40449258161665663		0.707828
Gradient Descent(679/999): loss=0.4044840589306599		0.70784
Gradient Descent(680/999): loss=0.40447556692601294		0.707836
Gradient Descent(681/999): loss=0.4044671054852464		0.707868
Gradient Descent(682/999): loss=0.40445867449139467		0.707852
Gradient Descent(683/999): loss=0.4044502738279949		0.707852
Gradient Descent(684/999): loss=0.4044419033790836		0.707848
Gradient Descent(685/999): loss=0.40443356302919276		0.707872
Gradient Descent(686/999): loss=0.4044252526633486		0.707884
Gradient Descent(687/999): loss=0.4044169721670684		0.707888
Gradient Descent(688/999): loss=0.40440872142635736		0.70792
Gradient Descent(689/999): loss=0.4044005003277067		0.707936
Gradient Descent(690/999): loss=0.40439230875809046		0.707944
Gradient Descent(691/999): loss=0.4043841466049631		0.707932
Gradient Descent(692/999): loss=0.4043760137562563		0.707944
Gradient Descent(693

Gradient Descent(812/999): loss=0.4035835714975051		0.707948
Gradient Descent(813/999): loss=0.4035782815879584		0.707976
Gradient Descent(814/999): loss=0.40357301025069		0.707972
Gradient Descent(815/999): loss=0.4035677574175542		0.707988
Gradient Descent(816/999): loss=0.4035625230206701		0.70798
Gradient Descent(817/999): loss=0.4035573069924219		0.707996
Gradient Descent(818/999): loss=0.403552109265456		0.708004
Gradient Descent(819/999): loss=0.40354692977268086		0.708
Gradient Descent(820/999): loss=0.4035417684472663		0.708012
Gradient Descent(821/999): loss=0.4035366252226417		0.708012
Gradient Descent(822/999): loss=0.40353150003249466		0.70802
Gradient Descent(823/999): loss=0.4035263928107704		0.708008
Gradient Descent(824/999): loss=0.4035213034916709		0.707992
Gradient Descent(825/999): loss=0.4035162320096528		0.708008
Gradient Descent(826/999): loss=0.40351117829942773		0.708004
Gradient Descent(827/999): loss=0.40350614229595955		0.708
Gradient Descent(828/999): loss

Gradient Descent(948/999): loss=0.40300994345958263		0.70796
Gradient Descent(949/999): loss=0.403006648915813		0.707928
Gradient Descent(950/999): loss=0.40300336571327694		0.707928
Gradient Descent(951/999): loss=0.40300009381142504		0.707928
Gradient Descent(952/999): loss=0.4029968331698593		0.707928
Gradient Descent(953/999): loss=0.4029935837483332		0.707916
Gradient Descent(954/999): loss=0.40299034550675034		0.707912
Gradient Descent(955/999): loss=0.4029871184051645		0.7079
Gradient Descent(956/999): loss=0.4029839024037786		0.707912
Gradient Descent(957/999): loss=0.40298069746294424		0.707876
Gradient Descent(958/999): loss=0.4029775035431613		0.707876
Gradient Descent(959/999): loss=0.40297432060507704		0.707872
Gradient Descent(960/999): loss=0.4029711486094857		0.707868
Gradient Descent(961/999): loss=0.4029679875173279		0.707864
Gradient Descent(962/999): loss=0.40296483728969		0.707884
Gradient Descent(963/999): loss=0.40296169788780395		0.707892
Gradient Descent(964/99

In [ ]:
w, loss = least_squares_SGD(y, x, w_init, 100, gamma=0.03)

In [ ]:
get_accuracy(x, y, w)

In [10]:
y_test, x_test, ids_test = load_csv_data(data_path="datas/test.csv", sub_sample=False)
pred_y = predict_labels(w, x_test)


In [11]:
create_csv_submission(ids_test, pred_y, "datas/submission.csv")
print('Done !')


Done !


In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

# https://stackoverflow.com/a/7941594/4810319
def main():
    np.random.seed(1977)
    numvars, numdata = 5, 100
    data = 10 * np.random.random((numvars, numdata))
    data = x[0:300, 0:7].T
    print(x[0:200, 7])
    fig = scatterplot_matrix(data, ['DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet'],
            linestyle='none', marker='o', color='black', mfc='none')
    fig.suptitle('Simple Scatterplot Matrix')
    plt.show()

def scatterplot_matrix(data, names, **kwargs):
    """Plots a scatterplot matrix of subplots.  Each row of "data" is plotted
    against other rows, resulting in a nrows by nrows grid of subplots with the
    diagonal subplots labeled with "names".  Additional keyword arguments are
    passed on to matplotlib's "plot" command. Returns the matplotlib figure
    object containg the subplot grid."""
    numvars, numdata = data.shape
    fig, axes = plt.subplots(nrows=numvars, ncols=numvars, figsize=(8,8))
    fig.subplots_adjust(hspace=0.05, wspace=0.05)

    for ax in axes.flat:
        # Hide all ticks and labels
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

        # Set up ticks only on one side for the "edge" subplots...
        if ax.is_first_col():
            ax.yaxis.set_ticks_position('left')
        if ax.is_last_col():
            ax.yaxis.set_ticks_position('right')
        if ax.is_first_row():
            ax.xaxis.set_ticks_position('top')
        if ax.is_last_row():
            ax.xaxis.set_ticks_position('bottom')

    # Plot the data.
    for i, j in zip(*np.triu_indices_from(axes, k=1)):
        for x, y in [(i,j), (j,i)]:
            axes[x,y].plot(data[x], data[y], **kwargs)

    # Label the diagonal subplots...
    for i, label in enumerate(names):
        axes[i,i].annotate(label, (0.5, 0.5), xycoords='axes fraction',
                ha='center', va='center')

    # Turn on the proper x or y axes ticks.
    for i, j in zip(range(numvars), itertools.cycle((-1, 0))):
        axes[j,i].xaxis.set_visible(True)
        axes[i,j].yaxis.set_visible(True)

    return fig

main()